In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.io 
import numpy as np
import pandas as pd
from scipy.signal import butter, lfilter
from scipy.fft import fft
import random
from sklearn import svm
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



In [2]:
faultI10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_10hz.mat')
faultI30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_1_30hz.mat')
faultII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_10hz.mat')
faultII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_2_30hz.mat')
faultIII10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_10hz.mat')
faultIII30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_3_30hz.mat')
faultIV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_10hz.mat')
faultIV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_4_30hz.mat')
faultV10=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_10hz.mat')
faultV30=scipy.io.loadmat('/home/sepehr/Desktop/Rotating Machine - Data/ZData_Fault_Rotating_Machine/unbalanced fault/without alphabet/Unbalanced.mat/Acquisition_un_5_30hz.mat')





In [3]:
sigII30=faultII30['Acquisition_un_2_30hz']
sigII10=faultII10['Acquisition_un_2_10hz']
del faultII30,faultII10
sigI30=faultI30['Acquisition_un_1_30hz']
sigI10=faultI10['Acquisition_un_1_10hz']
del faultI30,faultI10
sigIII10=faultIII10['Acquisition_un_3_10hz']
sigIII30=faultIII30['Acquisition_un_3_30hz']
del faultIII10,faultIII30
sigIV10=faultIV10['Acquisition_un_4_10hz']
sigIV30=faultIV30['Acquisition_un_4_30hz']
del faultIV10,faultIV30
sigV10=faultV10['Acquisition_un_5_10hz']
sigV30=faultV30['Acquisition_un_5_30hz']
del faultV10,faultV30
# sigVI10=faultVI10['Acquisition_un_6_10hz']
# sigVI30=faultVI30['Acquisition_un_6_30hz']
# del faultVI10,faultVI30

In [4]:

def sliding_window(data, window_size, stride):
    """
    Applies a sliding window to the given data with the specified window size
    and stride length. Padding is added at the beginning and end of the data to
    ensure all samples are included in windows.
    """
    num_padding = window_size - stride
    data_padded = np.pad(data, ((num_padding, num_padding), (0, 0)), 'constant')
    window_data = []
    for i in range(0, len(data_padded) - window_size + 1, stride):
        window = data_padded[i:i+window_size, :]
        window_data.append(window)
    window_data = np.array(window_data)
    # window_data = window_data[:training_size, :, :]
    window_data = np.transpose(window_data, (0, 2, 1))
    return window_data

In [5]:
windowsize=1500
stride=1400

In [6]:
sigtI10hz=sliding_window(sigI10,windowsize,stride)[:,1:13,:]
sigtI30hz=sliding_window(sigI30,windowsize,stride)[:,1:13,:]
sigtII10hz=sliding_window(sigII10,windowsize,stride)[:,1:13,:]
sigtII30hz=sliding_window(sigII30,windowsize,stride)[:,1:13,:]
del sigI10,sigI30,sigII10,sigII30
sigtIII30hz=sliding_window(sigIII30,windowsize,stride)[:,1:13,:]
sigtIII10hz=sliding_window(sigIII10,windowsize,stride)[:,1:13,:]
sigtIV30hz=sliding_window(sigIV30,windowsize,stride)[:,1:13,:]
sigtIV10hz=sliding_window(sigIV10,windowsize,stride)[:,1:13,:]
del sigIII30,sigIII10,sigIV30,sigIV10
sigtV30hz=sliding_window(sigV30,windowsize,stride)[:,1:13,:]
sigtV10hz=sliding_window(sigV10,windowsize,stride)[:,1:13,:]
# sigtVI30hz=sliding_window(sigVI30,windowsize,stride)
# sigtVI10hz=sliding_window(sigVI10,windowsize,stride)
# del sigV30,sigV10,sigVI30,sigVI10

In [7]:
datalable1=np.concatenate([sigtI10hz,sigtI30hz],axis=-1)

datalable2=np.concatenate([sigtII10hz,sigtII30hz],axis=-1)

datalable3=np.concatenate([sigtIII10hz,sigtIII30hz],axis=-1)

datalable4=np.concatenate([sigtIV10hz,sigtIV30hz],axis=-1)

datalabele5=np.concatenate([sigtV10hz,sigtV30hz],axis=-1)


In [8]:
labels=np.concatenate((np.ones(np.shape(datalable1)[0]),np.zeros(np.shape(datalable1)[0]),np.ones(np.shape(datalable1)[0])*4,np.ones(np.shape(datalable1)[0])*2,np.ones(np.shape(datalable1)[0])*3),axis=0)

dataall=np.concatenate([datalable1,datalable2,datalable3,datalable4,datalabele5],axis=0)

In [9]:
np.shape(dataall)

(2150, 12, 3000)

In [10]:
import numpy as np
from scipy.signal import stft

# generate example signal with shape (7530, 12, 1000)
data = dataall

# set STFT parameters
window = 'hann'
nperseg = 256
noverlap = 128
fs = 10000

# compute STFT spectrogram for each channel
spectrograms = []
for i in range(data.shape[1]):
    _, _, spec = stft(data[:, i, :], fs=fs, window=window, nperseg=nperseg, noverlap=noverlap)
    spectrograms.append(np.abs(spec))

# stack spectrograms along the channel axis to get shape (7530, 12, num_freq_bins, num_time_bins)
spectrograms = np.stack(spectrograms, axis=1)





In [11]:
np.shape(spectrograms)

(2150, 12, 129, 25)

In [12]:
mean = np.mean(spectrograms, axis=(0, 1, 2, 3))
std = np.std(spectrograms, axis=(0, 1, 2, 3))
normalized_data = (spectrograms - mean) / std
data=normalized_data


In [13]:
np.shape(data)

(2150, 12, 129, 25)

In [14]:
indices = np.random.permutation(data.shape[0])

# Use the shuffled indices to shuffle both data and labels
shuffled_data = data[indices]
shuffled_labels = labels[indices]
data= shuffled_data
labels=shuffled_labels


In [15]:
from sklearn.model_selection import train_test_split

# Assume stft_data is your STFT results stored in a variable
# with shape (7530, 12, 129, 9)

# Generate labels for your data (assuming you have labels available)


# Split the data and labels into training and test sets
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)


print(f"x_train shape: {x_train.shape}")  # should be (6024, 12, 129, 9)
print(f"y_train shape: {y_train.shape}")  # should be (6024,)
print(f"x_test shape: {x_test.shape}")    # should be (1506, 12, 129, 9)
print(f"y_test shape: {y_test.shape}")    # should be (1506,)


x_train shape: (1720, 12, 129, 25)
y_train shape: (1720,)
x_test shape: (430, 12, 129, 25)
y_test shape: (430,)


In [16]:
from keras.utils import to_categorical
y_train1 = to_categorical(y_train)
y_test1 = to_categorical(y_test, num_classes=5)
# print(f"x_test shape: {y_train1.shape}")    # should be (1506, 12, 129, 9)
# print(f"y_test shape: {y_test.shape}")

2023-05-03 20:49:38.388022: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 20:49:38.548030: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-03 20:49:38.549770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-03 20:49:39.749721: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
x_train = x_train.reshape((1720,12,129*25))
x_test = x_test.reshape((430,12,129*25))

In [18]:
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, concatenate
from tensorflow.keras.models import Model

input_shape = (12, 129*25)
num_classes = 5

# Define input layer
inputs = Input(shape=input_shape)

# First scale
conv1 = Conv1D(filters=32, kernel_size=3, activation='relu', padding='same')(inputs)
pool1 = MaxPooling1D(pool_size=2)(conv1)

# Second scale
conv2 = Conv1D(filters=64, kernel_size=3, activation='relu', padding='same')(pool1)
pool2 = MaxPooling1D(pool_size=2)(conv2)

# Third scale
conv3 = Conv1D(filters=128, kernel_size=3, activation='relu', padding='same')(pool2)
pool3 = MaxPooling1D(pool_size=2)(conv3)

# Four scale
# conv4 = Conv1D(filters=256, kernel_size=1, activation='relu', padding='same')(pool3)
# pool4 = MaxPooling1D(pool_size=1)(conv4)

# Flatten and concatenate scales
flatten = Flatten()(conv3)
dense1 = Dense(128, activation='relu')(flatten)
dropout1 = Dropout(0.7)(dense1)

# Add final output layer
outputs = Dense(num_classes, activation='softmax')(dropout1)

# Create model
model = Model(inputs=inputs, outputs=outputs)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train1, validation_split=0.2, epochs=150, batch_size=256)


2023-05-03 20:49:41.475146: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-03 20:49:41.476313: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Epoch 1/150


2023-05-03 20:49:41.892296: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 213004800 exceeds 10% of free system memory.


6/6 [==============================] - 3s 168ms/step - loss: 1.7779 - accuracy: 0.1890 - val_loss: 1.6104 - val_accuracy: 0.1831
Epoch 2/150
6/6 [==============================] - 1s 123ms/step - loss: 1.6162 - accuracy: 0.1999 - val_loss: 1.6135 - val_accuracy: 0.1715
Epoch 3/150
6/6 [==============================] - 1s 128ms/step - loss: 1.6130 - accuracy: 0.1962 - val_loss: 1.6088 - val_accuracy: 0.1715
Epoch 4/150
6/6 [==============================] - 1s 120ms/step - loss: 1.6024 - accuracy: 0.2267 - val_loss: 1.6047 - val_accuracy: 0.2442
Epoch 5/150
6/6 [==============================] - 1s 120ms/step - loss: 1.5973 - accuracy: 0.2420 - val_loss: 1.5977 - val_accuracy: 0.1715
Epoch 6/150
6/6 [==============================] - 1s 108ms/step - loss: 1.5890 - accuracy: 0.2565 - val_loss: 1.5823 - val_accuracy: 0.2297
Epoch 7/150
6/6 [==============================] - 1s 111ms/step - loss: 1.5720 - accuracy: 0.2711 - val_loss: 1.5666 - val_accuracy: 0.2413
Epoch 8/150
6/6 [========

In [19]:
import numpy as np
from sklearn.metrics import confusion_matrix


# Evaluate the model on the test set with multi-class labels
test_loss_multi, test_acc_multi = model.evaluate(x_test, y_test1, verbose=2)
print('Test accuracy multi:', test_acc_multi)

# Convert y_test1 to a multi-class format
y_pred_multi = model.predict(x_test)
y_test_pred_multi = np.argmax(y_pred_multi, axis=1)

# Compute the confusion matrix for multi-class
print('Multi-class:', confusion_matrix(np.argmax(y_test1, axis=1), y_test_pred_multi))

# Evaluate the model on the test set with normal labels
test_loss_normal, test_acc_normal = model.evaluate(x_test, y_test1, verbose=2)
print('Test accuracy normal:', test_acc_normal)

# Convert y_test to a multi-class format
y_pred_normal = model.predict(x_test)
y_test_pred_normal = np.argmax(y_pred_normal, axis=1)

# Compute the confusion matrix for normal labels
print('Normal labels:', confusion_matrix(y_test, y_test_pred_normal))


14/14 - 0s - loss: 0.0661 - accuracy: 0.9860 - 59ms/epoch - 4ms/step
Test accuracy multi: 0.9860464930534363
14/14 [==============================] - 0s 3ms/step
Multi-class: [[100   0   0   0   0]
 [  2  81   0   0   0]
 [  0   0  84   0   0]
 [  0   0   4  76   0]
 [  0   0   0   0  83]]
14/14 - 0s - loss: 0.0661 - accuracy: 0.9860 - 52ms/epoch - 4ms/step
Test accuracy normal: 0.9860464930534363
14/14 [==============================] - 0s 3ms/step
Normal labels: [[100   0   0   0   0]
 [  2  81   0   0   0]
 [  0   0  84   0   0]
 [  0   0   4  76   0]
 [  0   0   0   0  83]]
